In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import warnings 
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,cross_validate,GridSearchCV 
from statsmodels.tsa.stattools import acf
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")

df_holidays=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
df_oil=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv") 
df_stores=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
df_test=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")

In [ ]:
# Accuracy metrics
#both entries must be arrays. Both cannot be series

from statsmodels.tsa.stattools import acf
from sklearn.metrics import r2_score

def forecast_accuracy(actual, forecast):
       
    #mape = np.mean(100*np.abs(forecast - actual)/np.abs(actual))        # MAPE  (mean absolute percentage error)
    #me   = np.mean(forecast - actual)                                   # ME    (mean error)
    #mae  = np.mean(np.abs(forecast - actual))                           # MAE   (mean absolute error)
    #mpe  = np.mean((forecast - actual)/actual)                          # MPE   (mean percentage error)
    rmse = np.mean((forecast - actual)**2)**.5                          # RMSE  (root mean square error) 
    msle= np.mean((np.log(forecast+1) - np.log(actual+1))**2)           # MSLE  (mean square logarithmic error)
    #corr = np.corrcoef(forecast, actual)[0,1]                           # corr  (pearson R coefficient)
    rsquared = r2_score(actual,forecast)                                #R-squared
    
    #mins = np.amin(np.hstack([forecast[:,None], actual[:,None]]), axis=1)
    #maxs = np.amax(np.hstack([forecast[:,None], actual[:,None]]), axis=1)  
    #minmax = (1 - np.mean(mins/maxs)).round(3)                          # minmax
    
    #acf1 = acf(forecast-actual)[1]                                      # ACF1 (first order auto correlation order)

    results = ({
                'rmse':rmse,
                'msle':msle,
                'Rsquared':rsquared,
              })
    
    results = pd.DataFrame.from_dict(results,orient='index',columns=['value'])
    
    return results.round(3)

In [ ]:
##FAST WAY TO RUN NOTEBOOK

##Unique rows and creating a group to clasify
df_stores2=df_stores[['city','state','type','cluster']].drop_duplicates()
df_stores2['group']=df_stores2.reset_index().index+1
#Merge both datasets 
df=pd.merge(df, df_stores, on='store_nbr')
df['date']=pd.to_datetime(df ['date'])
df['day']=df['date'].dt.day_name()
#Merge both datasets to create the new column for the group
df=pd.merge(df, df_stores2, on=['city','state','type','cluster'])
#CREATING A NEW COLUMN FOR MONTH-DAY
df['month']=df['date'].dt.strftime('%m-%d')
##CREATE A NEW COLUMN FOR MONTH
df['month']=df['date'].dt.month
#CREATE A NEW COLUMN FOR PERIOD
df['period']=df['date'].dt.strftime('%y-%m')
##CREATE DATE AND DAY COLUMNS
df_holidays['date']=pd.to_datetime(df_holidays ['date'])
df_holidays=df_holidays[df_holidays['transferred']==False].drop(columns=['transferred'])
df_holidays.rename(columns={'type':'type_day'},inplace=True)
#Merge both datasets
df=df.merge(df_holidays,how='left', left_on='date', right_on='date')
df['type_day'].fillna('Normal',inplace=True)
#Merge both datasets
df_oil['date']=pd.to_datetime(df_oil ['date'])
df=df.merge(df_oil,how='left', left_on='date', right_on='date')
#CREATE A NEW COLUMN FOR DAY OF WEEK
df['day']=df['date'].dt.dayofweek
#CREATE A NEW COLUMN FOR DAY NAME
df['day_name']=df['date'].apply(lambda x: x.strftime('%A'))

In [ ]:
#Removing noisy data

df2=df.copy()
disaster = df2[(df2['date'] >= '2016-04-16') & (df2['date'] <= '2016-10-16')]
disaster.sort_values('sales', ascending = False)
df =df.drop(disaster.index)

In [ ]:
#for the test set
df_test=pd.merge(df_test, df_stores, on='store_nbr')
df_test['date']=pd.to_datetime(df_test ['date'])
df_test['day']=df_test['date'].dt.day_name()
#Merge both datasets to create the new column for the group
df_test=pd.merge(df_test, df_stores2, on=['city','state','type','cluster'])
#CREATING A NEW COLUMN FOR MONTH-DAY
df_test['month']=df_test['date'].dt.strftime('%m-%d')
##CREATE A NEW COLUMN FOR MONTH
df_test['month']=df_test['date'].dt.month
#CREATE A NEW COLUMN FOR PERIOD
df_test['period']=df_test['date'].dt.strftime('%y-%m')
##CREATE DATE AND DAY COLUMNS
df_holidays['date']=pd.to_datetime(df_holidays ['date'])
#Merge both datasets
df_test=df_test.merge(df_holidays,how='left', left_on='date', right_on='date')
df_test['type_day'].fillna('Normal',inplace=True)
#Merge both datasets
df_test=df_test.merge(df_oil,how='left', left_on='date', right_on='date')
#CREATE A NEW COLUMN FOR DAY OF WEEK
df_test['day']=df_test['date'].dt.dayofweek
#CREATE A NEW COLUMN FOR DAY NAME
df_test['day_name']=df_test['date'].apply(lambda x: x.strftime('%A'))

In [ ]:
#RANDOM FOREST REGRESSOR FOR EACH GROUP



test=[]
pred=[]
models=[]

groups=np.sort(df['group'].unique())

for group in groups:
    
    dff=df[df['group']==group]
    
    encoder_family = LabelEncoder()
    dff['family']=encoder_family.fit_transform(dff['family'])
    
    encoder_day = LabelEncoder()
    dff['day']=encoder_day.fit_transform(dff['day'])
    
    data=dff[['onpromotion','family','month','day']]
    target=dff['sales']
    
    x_train, x_test, y_train, y_test = train_test_split(data, target, train_size = 0.8, random_state = 5)
    x_train = pd.get_dummies(x_train, drop_first=True)
    x_test = pd.get_dummies(x_test, drop_first=True)
    regr  = RandomForestRegressor(n_estimators = 10, random_state = 1)
    regr.fit(x_train, y_train) 
    
    models.append (regr)
    
    y_pred=regr.predict(x_test)
    
    a=forecast_accuracy(y_test, y_pred)
    msle=a.loc['msle'][0]
    
    test.extend(y_test)
    pred.extend(y_pred)
    print(f"modelo {group} terminado y msle de {msle} ")

In [ ]:
pred=[]

groups=np.sort(df['group'].unique())

for group in groups:
    
    dff=df_test[df_test['group']==group]
    encoder_family = LabelEncoder()
    dff['family']=encoder_family.fit_transform(dff['family'])
       
    encoder_day = LabelEncoder()
    dff['day']=encoder_day.fit_transform(dff['day'])


    data=dff[['onpromotion','family','month','day']]
    data = pd.get_dummies(data, drop_first=True)
    
    i= group-1
    
    y_pred=models[i].predict(data)
    pred.extend(y_pred)    
pred[0:5]

In [ ]:
df_test

In [ ]:
output = pd.DataFrame({'id': df_test['id'], 'sales': pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")